In [ ]:
from pathlib import Path
import h5py
from typing import Tuple, Dict
import numpy as np

SIM_PATH_LEGACY = Path("../data/results/salida_modelo_copy.h5")
print(SIM_PATH_LEGACY.resolve())  # Te muestra la ruta absoluta
print(SIM_PATH_LEGACY.exists())  # True si existe, False si no

In [ ]:
def load_all_datasets(path: str | Path) -> Dict[str, np.ndarray]:
    """Carga todos los datasets de un archivo HDF5 plano (sin grupos)."""
    arrays: Dict[str, np.ndarray] = {}
    with h5py.File(path, "r") as f:
        for name, ds in f.items():
            if isinstance(ds, h5py.Dataset):
                arrays[name] = ds[:]  # pasar a numpy
    return arrays


# Abrir archivo en modo lectura
with h5py.File(SIM_PATH_LEGACY.resolve(), "r") as f:
    # Listar los grupos principales
    for key in f.keys():
        print(f[key])

In [ ]:
data: Dict[str, np.ndarray] = load_all_datasets(SIM_PATH_LEGACY.resolve())

In [ ]:
print(data.keys())

In [ ]:
import h5py

mapping = {
    "EUR": "eur_act",
    "biomasaAcumulada": "biomass_cum",
    "biomasaDiaria": "biomass_daily",
    "cEstresHidrico": "ceh",
    "cEstresHidricoRadiacion": "ceh_r",
    # "ceh_pc": "ceh_pc",
    "evaporacion": "soil_evap",
    "porcAguaUtil": "p_au",
    "porcCobertura": "cover",
    "precipitacion_efectiva": "eff_precip",
    "profRaiz": "root_depth",
    "rendimiento": "yield_tensor",
    "transpiracion": "transpiration",
}


def rename_h5_keys(file_path: Path, mapping: dict[str, str]) -> None:
    """Renames datasets or groups in an HDF5 file based on a mapping.

    Parameters
    ----------
    file_path : str
        Path to the .h5 file to modify.
    mapping : dict[str, str]
        Dictionary mapping old keys to new keys.
    """
    with h5py.File(file_path, "r+") as f:
        for old_key, new_key in mapping.items():
            if old_key in f:
                print(f"Renaming '{old_key}' → '{new_key}'")
                f.copy(f[old_key], new_key)
                del f[old_key]
            else:
                print(f"Key '{old_key}' not found — skipped.")

In [ ]:
rename_h5_keys(SIM_PATH_LEGACY.resolve(), mapping)

In [ ]:
data = load_all_datasets(SIM_PATH_LEGACY.resolve())
print(data.keys())

In [ ]:
keys_to_drop = [
    "ici",
    "drenaje",
    "diasDesdePrec",
    "percolacion_capa_1",
    "percolacion_capa_2",
    "percolacion_capa_3",
    "aguaUtilDisp1",
    "aguaUtilDisp2",
    "aguaUtilDisp3",
    "aguaUtilDisp4",
    "cEstrsTerminco",
]

In [ ]:
filtered = {k: v for k, v in data.items() if k not in keys_to_drop}

In [ ]:
for key in filtered.keys():
    print(key, filtered[key].shape)
    print(filtered[key][0:5, :5, :5])

    # filtered_reordered = np.transpose(filtered[key], (2, 1, 0))
    # print(filtered[key].T[0, :3, :10])
    # print()

    # print(key, filtered_reordered.shape)
    # print(filtered_reordered[:3, :10, 0])
    # print()

In [80]:
filtered["p_au"][0:2, 0:5, 0:5]

array([[[       nan, 0.86666667, 0.86666667, 0.86666667, 0.86666667],
        [       nan,        nan, 0.86666667, 0.86666667, 0.86666667],
        [       nan,        nan,        nan, 0.86666667, 0.86666667],
        [       nan,        nan,        nan, 0.86666667, 0.86666667],
        [       nan,        nan,        nan, 0.86666667,        nan]],

       [[       nan, 0.85227121, 0.85227121, 0.85227121, 0.85227121],
        [       nan,        nan, 0.85227121, 0.85227121, 0.85227121],
        [       nan,        nan,        nan, 0.85227121, 0.85227121],
        [       nan,        nan,        nan, 0.85227121, 0.85227121],
        [       nan,        nan,        nan, 0.85227121,        nan]]])

In [81]:
np.transpose(filtered["p_au"], (0, 2, 1))[0:2, 0:5, 0:5]

array([[[       nan,        nan,        nan,        nan,        nan],
        [0.86666667,        nan,        nan,        nan,        nan],
        [0.86666667, 0.86666667,        nan,        nan,        nan],
        [0.86666667, 0.86666667, 0.86666667, 0.86666667, 0.86666667],
        [0.86666667, 0.86666667, 0.86666667, 0.86666667,        nan]],

       [[       nan,        nan,        nan,        nan,        nan],
        [0.85227121,        nan,        nan,        nan,        nan],
        [0.85227121, 0.85227121,        nan,        nan,        nan],
        [0.85227121, 0.85227121, 0.85227121, 0.85227121, 0.85227121],
        [0.85227121, 0.85227121, 0.85227121, 0.85227121,        nan]]])

In [ ]:
transposed = np.transpose(filtered["p_au"], (0, 2, 1))[0:2, 0:5, 0:5]

In [ ]:
np.transpose(transposed)